# TP0 - Exercício 2 - Grupo 15
#### João Esteves - pg46535
#### Sara Queirós - pg47661

2. Criar uma cifra com autenticação de meta-dados a partir de um PRG

    a. Criar um gerador pseudo-aleatório do tipo XOF (“extened output function”) usando o SHAKE256, para gerar uma sequência de palavras de 64 bits. 
     1. O gerador deve poder gerar até um limite de 2^n palavras (n é  um parâmetro) armazenados em long integers do Python.
     2. A “seed” do gerador funciona como cipher_key e é gerado por um KDF a partir de uma “password” .
     3. A autenticação do criptograma e dos dados associados é feita usando o próprio SHAKE256.
   
    b. Defina os algoritmos de cifrar e decifrar : para cifrar/decifrar uma mensagem com blocos de 64 bits, os “outputs” do gerador são usados como máscaras XOR dos blocos da mensagem. 
    Essencialmente a cifra básica é uma implementação do  “One Time Pad”.


In [ ]:
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import cryptography.exceptions
from cryptography.hazmat.primitives import padding

## Criar gerador de palavras 
Para derivar uma chave, ou seja, a cipher_key, com 32 bytes usamos uma Key Derivation Function (utilizamos a PBKDF2HMAC fornecida pela biblioteca cryptography), com uma password dada como input, utilizando um salt aleatório.

In [ ]:
def get_seed(self, password):
        #gerar a seed para o PRG usando KDF
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=self.salt,
            iterations=390000,)
        seed = kdf.derive(bytes(password, 'utf-8'))
        return seed

Obtida a seed, o Pseudo Random Generator (função PRG), utiliza o SHAKE256 para gerar N palavras, para N dado como parâmetro. Uma vez que não existem long integers em python3, utilizamos uma lista de strings para armazenar as palavras geradas. 

In [ ]:
def PRG(self, seed, n):
        #64 bites = 8 blocos de 1 byte(8 bits)
        #gerador com limite de 2^n palavras 
        dig = hashes.Hash(hashes.SHAKE256(8 * pow(2,n)))
        dig.update(seed)
        rand = dig.finalize()
        self.words = rand

## Autenticação 
* Para verificar a autenticidade da mensagem, tentamos utilizar o SHAKE256. No entanto, como dava erro relacionado com o OpenSSL, utilizamos o SHA256 para o efeito, com as words geradas pelo PRG.
* Isto é acompanhado por verificação por parte do Receiver relativamente à autenticidade da mensagem que está a receber.


In [ ]:
def authentication(self, message):
        h = hmac.HMAC(self.words, hashes.SHA256())
        h.update(message)
        return h.finalize()

## Cifrar a mensagem
* Para cifrar a mensagem, como uma implementação do One Time Pad, é necessário fazer o padding do input que se pretende. Também é necessário verificar se o tamanho da mensagem não é superior ao do output do gerador, pois, caso não seja, não consegue cifrar/decifrar o pretendido. 
* Para além disso, por se tratarem de blocos de 64 bits(8 bytes), foi feita a divisão da mensagem e aplicados os outputs do gerador como máscaras XOR ao blocos da mensagem. Desta forma obtém-se o texto cifrado.

In [ ]:
def encode(self, data):
        #fazer o padding do texto 
        padder = padding.PKCS7(64).padder()
        padded = padder.update(data) + padder.finalize()
        #verificar se a chave tem tamanho suficiente para cifrar o texto dado como input 
        if (len(self.words) < len(padded)):
            raise Exception("O tamanho da chave não é suficiente!")        
        else:
            cipher_text = b''
            #cifrar com as words geradas pelo PRG
            for i in range (0,len(padded),8):
                p = padded[i:i+8]
                for index, block in enumerate(p):   
                    #aplicar as máscaras XOR aos blocos
                    cipher_text += bytes([block ^ self.words[i*8:(i+1)*8][index]])
            return cipher_text     

## Decifrar a mensagem
Para decifrar a mensagem utilizamos exatamente a mesma estratégia de a cifrar. No final é feito o unpadding com o intuito de remover os caracteres indesejados que foram adicionados no final da mensagem para completar o último bloco.

In [ ]:
def decode (self, ciphertext):
        plaintext = b''
        #aplica a mesma função de encode ao ciphertext
        for i in range (0,len(ciphertext),8):
            p = ciphertext[i:i+8]
            for index, block in enumerate(p):   
                #aplicar as máscaras XOR aos blocos
                plaintext += bytes([block ^ self.words[i*8:(i+1)*8][index]])   
        #faz o unpadded para remover os caracteres adicionados 
        unpadder = padding.PKCS7(64).unpadder()
        unpadded = unpadder.update(plaintext) + unpadder.finalize()
        return unpadded.decode('utf-8')

Para enviar e receber a mensagem foram utilizadas essencialmente as mesmas funções do exercício 1.

* Apresenta-se em baixo um exemplo do resultado dado para a mensagem "Mensagem utilizada para teste", com n = 10:
![](../img/ex2_1.png)



* Outro exemplo para "Segunda mensagem de teste", com n = 12:
![](../img/ex2_2.png)